In [ ]:
# eval_langchain_agent.py
import os, time, re, statistics
import pandas as pd
from pydantic import BaseModel, Field

from langchain_deepseek import ChatDeepSeek
from langchain_experimental.tools import PythonAstREPLTool
from langchain.agents import create_agent
from langchain.tools import tool

# ----------------------
# 1) data
# ----------------------
telco = pd.read_csv("telco_data.csv")

ast = PythonAstREPLTool(locals={"telco": telco})
# Removed the problematic guard invocation block here.
# The guard function itself is fine, but its global invocation with `sr.search_formula` was incorrect.

# ----------------------
# 2) schemas (same spirit as your notebook)
# ----------------------
class WriteFormulaFormat(BaseModel):
    search_formula: str = Field(..., description="Only python code using pandas/builtins. Return code only.")

class ResFormat(BaseModel):
    query: str
    formula: str
    result: str

# ----------------------
# 3) model
# ----------------------
model = ChatDeepSeek(
    model="deepseek-chat",
    api_key="sk-ff600c0cbe524d80a76c5c9d0b7c17b0", # Directly using the API key string
    base_url=os.getenv("DEEPSEEK_BASE_URL", "https://api.deepseek.com"),
)

# ----------------------
# 4) tool: write_formula (same pattern as notebook)
# ----------------------
@tool
def write_formula(query: str) -> str:
    """根据用户查询生成用于计算的 Python 代码。"""
    system_prompt = """
你是一位专业的数据分析师：
1) 你可以访问一个名为 telco 的 pandas DataFrame
2) 只返回用于计算的 Python 代码（pandas + 内置库），不要输出解释文字
"""
    f_agent = create_agent(
        model=model,
        tools=[ast],  # 可让模型先用 ast 看 telco.head()
        system_prompt=system_prompt,
        response_format=WriteFormulaFormat,
    )
    conv = [{"role": "user", "content": query}]
    r = f_agent.invoke({"messages": conv})
    return r["structured_response"].search_formula

# ----------------------
# 5) main agent
# ----------------------
system_prompt = """
你是数据分析助手：
- 先生成可执行的 pandas 代码
- 再调用 ast 执行并返回结果
- 输出必须符合结构化 schema（query/formula/result）

你是一位专业的数据分析师。

你只能使用当前环境中已经存在的 pandas DataFrame：telco。
严格禁止以下行为：
- 禁止读取或写入任何文件（例如 pd.read_csv / to_csv / open 等）
- 禁止创建新的 DataFrame 来替代 telco（禁止 telco = ...，禁止 pd.DataFrame(...)）
- 禁止生成示例数据或修改 telco 的内容/列名/行数

你的任务：把用户问题转成可执行的 pandas 代码。
要求：
1) 只返回 Python 代码（不要解释文字、不要 markdown）
2) 代码必须以 telco 为输入，只做查询/聚合/过滤
3) 最终输出必须是一个标量数值（float 或 int），便于后续解析
"""
agent = create_agent(
    model=model,
    tools=[ast, write_formula],
    system_prompt=system_prompt,
    response_format=ResFormat,
)

# ----------------------
# 6) test set (all verifiable by pandas)
# ----------------------
TESTS = [
    ("计算 MonthlyCharges 的均值", lambda df: float(df["MonthlyCharges"].mean())),
    ("计算 MonthlyCharges 的中位数", lambda df: float(df["MonthlyCharges"].median())),
    ("计算 tenure 的均值", lambda df: float(df["tenure"].mean())),
    ("计算 SeniorCitizen 的平均值（占比）", lambda df: float(df["SeniorCitizen"].mean())),
    ("计算 TotalCharges 的均值（忽略无法转成数值的）",
     lambda df: float(pd.to_numeric(df["TotalCharges"], errors="coerce").mean())),
    ("计算每个 Contract 的平均 MonthlyCharges，然后返回最大值",
     lambda df: float(df.groupby("Contract")["MonthlyCharges"].mean().max())),
    ("计算 Churn='Yes' 的比例",
     lambda df: float((df["Churn"] == "Yes").mean())),
    ("计算 InternetService='Fiber optic' 的比例",
     lambda df: float((df["InternetService"] == "Fiber optic").mean())),
    ("计算 PaymentMethod 的类别数",
     lambda df: float(df["PaymentMethod"].nunique())),
    ("计算 gender='Female' 的比例",
     lambda df: float((df["gender"] == "Female").mean())),
]

def to_float(x: str):
    # 从 result 字符串里尽量抽取数值
    m = re.findall(r"-?\d+(?:\.\d+)?", str(x))
    return float(m[0]) if m else None

def run_once(prompt: str):
    t0 = time.perf_counter()
    try:
        r = agent.invoke({"messages": [{"role": "user", "content": prompt}]})
        dt = time.perf_counter() - t0
        sr = r.get("structured_response", None)
        schema_ok = sr is not None and all(getattr(sr, k, None) is not None for k in ["query","formula","result"])
        return dt, schema_ok, sr
    except Exception as e:
        dt = time.perf_counter() - t0
        return dt, False, None

latencies, schema_ok_cnt, exec_ok_cnt, correct_cnt = [], 0, 0, 0

for q, gt_fn in TESTS:
    expected = gt_fn(telco)
    dt, schema_ok, sr = run_once(q)
    latencies.append(dt)
    if schema_ok:
        schema_ok_cnt += 1
        got = to_float(sr.result)
        if got is not None:
            exec_ok_cnt += 1
            # 数值容忍度（可按需要调）
            if abs(got - expected) <= max(1e-6, 1e-3 * max(1.0, abs(expected))):
                correct_cnt += 1

N = len(TESTS)
def pct(a): return 100.0 * a / N

p50 = statistics.median(latencies)
p95 = sorted(latencies)[int(0.95*(N-1))]

print(f"Data: {telco.shape[0]} rows x {telco.shape[1]} cols")
print(f"N={N}")
print(f"Schema adherence: {pct(schema_ok_cnt):.1f}%")
print(f"Tool execution (numeric parsed): {pct(exec_ok_cnt):.1f}%")
print(f"Correctness (vs pandas): {pct(correct_cnt):.1f}%")
print(f"Latency: p50={p50:.2f}s, p95={p95:.2f}s")

TotalCharges列的数据类型: object

前10行数据:
0      29.85
1     1889.5
2     108.15
3    1840.75
4     151.65
5      820.5
6     1949.4
7      301.9
8    3046.05
9    3487.95
Name: TotalCharges, dtype: object
TotalCharges的均值（忽略无法转换的值）: 2283.3004408418656
有效数据数量: 7032
telco DataFrame shape: (7043, 22)
telco columns: ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn', 'TotalCharges_numeric']
telco head:
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOC

In [ ]:
pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
pip install langchain-deepseek

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.3 MB/s eta 0:00:00
